In [2]:
import logging

import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..'))
sys.path.append(root_dir)

import numpy as np
import pandas as pd

from importlib import reload

from memory import garbage_collect

log = lambda msg: logging.info(msg)


# Load data

In [32]:
from sleep_events import load_days_data

day_data = load_days_data(False)

In [33]:
df = day_data.copy()

# Data prep - shared

In [34]:
import notebooks.DayData.day_data_processing as day_data_processing
reload(day_data_processing)


groupings = day_data_processing.group_columns_by_prefix(day_data)
groupings


,prefix,column_count,present_count,missing_count,missing_%
0,dayAndNightOf,1,359,0,0.0
1,date,6,2154,0,0.0
2,weather,11,3949,0,0.0
3,night,1061,130794,250105,65.7
4,sunrise,8,2872,0,0.0
5,food,263,61731,32686,34.6
6,drugsAndSupplements,139,27393,22508,45.1
7,exercise,87,20818,10415,33.3
8,phoneUsage,217,45927,31976,41.0
9,position,42,4842,10236,67.9


In [42]:
import notebooks.Util.Time as Time
reload(Time)

df_shared = Time.convert_cols_to_hours(df)


c:\dev\Brainwave-Processor\notebooks\Util\Time.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col].apply(
c:\dev\Brainwave-Processor\notebooks\Util\Time.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col].apply(
c:\dev\Brainwave-Processor\notebooks\Util\Time.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [47]:
import notebooks.DayData.day_data_processing as day_data_processing
reload(day_data_processing)

cols_before = len(df_shared.columns)
cols_to_drop = [c for c in df_shared.columns if day_data_processing.isMlKeyUselessOrDeprecated(c)]
df_shared.drop(columns=cols_to_drop, inplace=True)
cols_after = len(df_shared.columns)
print(f"Dropped {cols_before - cols_after} columns")
df_shared.head()

Dropped 465 columns


,dayAndNightOf,date:daysSince,date:dayOfWeek,date:dayOfMonth,date:month,date:year,date:asStr,weather:cloudCoverAfternoon,weather:humidityAfternoon,weather:precipitationTotal,...,events:luminetteLevel2:firstHSM,events:luminetteLevel2:lastHSM,events:sex-with-climax:firstHSM,events:sex-with-climax:lastHSM,drugsAndSupplements:MultiVitFirstHSM,drugsAndSupplements:MultiVitLastHSM,drugsAndSupplements:MelatoninXRFirstHSM,drugsAndSupplements:MelatoninXRLastHSM,sunExposure:firstEnteredOutsideHSM,sunExposure:lastOutsideHSM
0,2025-02-03,0,1,3,2,2025,2025-02-03,20,77,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-21,0,7,21,4,2024,2024-04-21,40,56,2.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-22,0,1,22,4,2024,2024-04-22,75,95,4.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-23,0,2,23,4,2024,2024-04-23,40,69,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-24,0,3,24,4,2024,2024-04-24,40,61,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Subset pipeline
A subset of data that we know is fairly clean and easy to work with

In [67]:
safe_list = [c for c in df_shared.columns if 'weather' in c]
df_subset = df_shared[safe_list]


import notebooks.Util.Data as Data
reload(Data)

missing_info = Data.analyze_missing_values(df_subset, df_subset.columns)
missing_info


,Present Count,Missing Count,Missing %,Type
weather:cloudCoverAfternoon,359,0,0.0,int64
weather:humidityAfternoon,359,0,0.0,int64
weather:precipitationTotal,359,0,0.0,float64
weather:temperatureMin,359,0,0.0,float64
weather:temperatureMax,359,0,0.0,float64
weather:temperatureAfternoon,359,0,0.0,float64
weather:temperatureNight,359,0,0.0,float64
weather:temperatureEvening,359,0,0.0,float64
weather:temperatureMorning,359,0,0.0,float64
weather:pressureAfternoon,359,0,0.0,int64


# Data pipeline - full

In [70]:
df.select_dtypes(include=['datetime64[ns, Europe/London]']).columns


Index([], dtype='object')

# No missing values pipeline

In [48]:
import notebooks.DayData.day_data_processing as day_data_processing
reload(day_data_processing)


df_no_missing = df_shared.copy()

import notebooks.Util.Data as Data
reload(Data)

missing_info = Data.analyze_missing_values(df_no_missing, df_no_missing.columns)
missing_info





,Present Count,Missing Count,Missing %,Type
vrUsage:categories:Other,0,359,100.0,float64
night:morningQuestionnaire,0,359,100.0,float64
energy,0,359,100.0,float64
settledScore,0,359,100.0,float64
night:aggregated:sleepScoreV2,0,359,100.0,float64
...,...,...,...,...
date:year,359,0,0.0,int64
date:dayOfMonth,359,0,0.0,int64
date:dayOfWeek,359,0,0.0,int64
date:daysSince,359,0,0.0,int64


In [18]:
low_missing_cols

['weather:pressureAfternoon',
 'weather:windMaxSpeed',
 'date:dayOfWeek',
 'drugsAndSupplements:RybelsusCount',
 'date:month',
 'date:dayOfMonth',
 'date:year',
 'dayAndNightOf',
 'drugsAndSupplements:GabapentinQuantity',
 'drugsAndSupplements:GabapentinCount',
 'drugsAndSupplements:GabapentinGabasignQuantity',
 'drugsAndSupplements:GabapentinGabasignCount',
 'drugsAndSupplements:GabapentinGabantinQuantity',
 'drugsAndSupplements:GabapentinGabantinCount',
 'drugsAndSupplements:GlycineQuantity',
 'drugsAndSupplements:GlycineCount',
 'drugsAndSupplements:SuperfoodPowderQuantity',
 'drugsAndSupplements:SuperfoodPowderCount',
 'drugsAndSupplements:ProteinPowderQuantity',
 'drugsAndSupplements:ProteinPowderCount',
 'drugsAndSupplements:ConcertaXLQuantity',
 'drugsAndSupplements:ConcertaXLCount',
 'drugsAndSupplements:Omega3Quantity',
 'exercise:byExercise:Run:totalSteps',
 'exercise:byExercise:Run:averageHeartRate',
 'drugsAndSupplements:TenPMSleepyQuantity',
 'drugsAndSupplements:TenPMSlee

In [26]:
# find row where index =  night:yasaExtended:Stability:Fpz:AnyStage
missing_info[missing_info.index == 'night:yasaExtended:Stability:Fpz:AnyStage']



,Present Count,Missing Count,Missing %,Type
night:yasaExtended:Stability:Fpz:AnyStage,1,358,99.7,float64


In [49]:
# Just drop any rows where < 5% of values are missing.  Saves time hand-treating ti all.
rows_before = len(df_no_missing)
low_missing_cols = missing_info[missing_info["Missing %"] < 5.0].index.tolist()
# low_missing_cols
[c for c in low_missing_cols if 'night:yasaExtended:Stability:Fpz:AnyStage' in c]
df_no_missing = df_no_missing.dropna(subset=low_missing_cols)
rows_after = len(df_no_missing)
print(f"Dropped {rows_before - rows_after} rows of {rows_before} due to missing a small amount of data")


Dropped 7 rows of 359 due to missing a small amount of data


In [30]:
missing_info = Data.analyze_missing_values(df_no_missing, df_no_missing.columns)
display(missing_info.describe())
missing_info


,Present Count,Missing Count,Missing %
count,2704.000000,2704.000000,2704.000000
mean,131.284024,220.715976,62.698706
std,105.385882,105.385882,29.940157
min,0.000000,0.000000,0.000000
25%,33.750000,174.000000,49.400000
50%,114.000000,238.000000,67.600000
75%,178.000000,318.250000,90.375000
max,352.000000,352.000000,100.000000


,Present Count,Missing Count,Missing %,Type
events:massageGunEnd:count,0,352,100.0,float64
vrUsage:categories:ActiveGame,0,352,100.0,float64
events:massageGunEnd:firstSSM,0,352,100.0,float64
events:massageGunEnd:lastSSM,0,352,100.0,float64
vrUsage:categories:Fitness,0,352,100.0,float64
...,...,...,...,...
weather:temperatureAfternoon,352,0,0.0,float64
weather:temperatureEvening,352,0,0.0,float64
weather:temperatureNight,352,0,0.0,float64
weather:humidityAfternoon,352,0,0.0,int64


In [50]:
import notebooks.Util.Data as Data
reload(Data)

remove_missing_rows_sequentially = Data.remove_missing_rows_greedy(df_no_missing, df_no_missing.columns)
remove_missing_rows_sequentially







Greedy removal:   0%|          | 0/2233 [00:00<?, ?it/s]

,Column,Removed,Removed %,Cumulative Removed,Cumulative Removed %
0,drugsAndSupplements:ProteinPowderCount,0,0.0,0,0.0
1,drugsAndSupplements:GabapentinGabasignCount,0,0.0,0,0.0
2,exercise:byExercise:Any:averageHeartRate,0,0.0,0,0.0
3,drugsAndSupplements:RitalinCount,0,0.0,0,0.0
4,drugsAndSupplements:MagnesiumThreonateQuantity,0,0.0,0,0.0
...,...,...,...,...,...
2228,drugsAndSupplements:MelatoninLastHSM,0,0.0,352,100.0
2229,circadian:combined:entries:MP2:minsUntilWake,0,0.0,352,100.0
2230,circadian:basic:entries:MP2:prominence,0,0.0,352,100.0
2231,events:startingPaidWork:count,0,0.0,352,100.0


In [64]:
import notebooks.Util.Data as Data
reload(Data)

over_50 = remove_missing_rows_sequentially[remove_missing_rows_sequentially['Cumulative Removed %'] > 50]
over_50.index = over_50['Column']
Data.group_rows_by_prefix(over_50).sort_values('present_count', ascending=False)

,prefix,row_count,present_count,missing_count,missing_%
2,night,551,2755,0,0.0
3,circadian,330,1650,0,0.0
10,phoneUsage,131,655,0,0.0
14,events,109,545,0,0.0
1,food,97,485,0,0.0
7,drugsAndSupplements,70,350,0,0.0
5,o2,56,280,0,0.0
4,coreBodyTemp,56,280,0,0.0
12,hrv,56,280,0,0.0
13,hr,56,280,0,0.0
